In [ ]:
import onnxruntime as ort
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import torch
from torchvision import transforms

# 定义颜色映射（BGR）
class_colors = {
    0: (0, 0, 0),
    1: (0, 255, 255),
    2: (0, 255, 0),
    3: (255, 0, 0),
    4: (0, 128, 0),
}

def decode_segmentation(segmentation, class_colors):
    h, w = segmentation.shape
    color_image = np.zeros((h, w, 3), dtype=np.uint8)
    for class_id, color in class_colors.items():
        color_image[segmentation == class_id] = color
    return color_image[:, :, ::-1]  # BGR -> RGB

def preprocess_image(image_path, input_size=(512, 1024)):
    img = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    img_tensor = transform(img).unsqueeze(0).numpy()
    return img_tensor, np.array(img)

# 加载 ONNX 模型
onnx_path = "bisenetv2_5_cls_v2.onnx"
session = ort.InferenceSession(onnx_path, providers=['CPUExecutionProvider'])

# 合并图像与 GT mask 路径
merged_image_path = "merged_images.jpg"
gt_mask_path = "merged_masks.png"

# --- 合并图像推理 ---
input_tensor, merged_rgb = preprocess_image(merged_image_path)
ort_out = session.run(None, {session.get_inputs()[0].name: input_tensor})
merged_pred_mask = ort_out[0][0, 0, :, :].astype(np.uint8)
merged_pred_color = decode_segmentation(merged_pred_mask, class_colors)

# GT mask 解码
gt_mask = cv2.imread(gt_mask_path, cv2.IMREAD_GRAYSCALE)
gt_mask = cv2.resize(gt_mask, (merged_pred_mask.shape[1], merged_pred_mask.shape[0]), interpolation=cv2.INTER_NEAREST)
merged_gt_color = decode_segmentation(gt_mask, class_colors)

# 单独图像路径
image_paths = [
    'segdata/images/test/2024-10-02_data_cleansing_9.jpg',
    'segdata/images/test/2024-10-02_data_cleansing_24.jpg',
    'segdata/images/test/2024-10-02_data_cleansing_37.jpg',
    'segdata/images/test/2024-10-02_data_cleansing_38.jpg',
]

# 保存单张图像及其预测结果
single_images = []
single_preds = []

for path in image_paths:
    input_tensor, raw_img = preprocess_image(path)
    ort_out = session.run(None, {session.get_inputs()[0].name: input_tensor})
    pred_mask = ort_out[0][0, 0, :, :].astype(np.uint8)
    pred_color = decode_segmentation(pred_mask, class_colors)

    single_images.append(raw_img)
    single_preds.append(pred_color)

# --- 布局可视化 ---
fig = plt.figure(figsize=(20, 12))

# Row 1: 合并图 + label + 预测
plt.subplot(3, 4, 1)
plt.imshow(merged_rgb)
plt.title("Merged Image")
plt.axis('off')

plt.subplot(3, 4, 2)
plt.imshow(merged_gt_color)
plt.title("Merged Ground Truth")
plt.axis('off')

plt.subplot(3, 4, 3)
plt.imshow(merged_pred_color)
plt.title("Merged Prediction")
plt.axis('off')

# 空占位保持对齐
plt.subplot(3, 4, 4)
plt.axis('off')

# Row 2: 原图 1~4
for i in range(4):
    plt.subplot(3, 4, 5 + i)
    plt.imshow(single_images[i])
    plt.title(f"Image {i+1}")
    plt.axis('off')

# Row 3: 推理结果 1~4
for i in range(4):
    plt.subplot(3, 4, 9 + i)
    plt.imshow(single_preds[i])
    plt.title(f"Prediction {i+1}")
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import onnxruntime as ort
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import torch
from torchvision import transforms

# 类别颜色（BGR）
class_colors = {
    0: (0, 0, 0),
    1: (0, 255, 255),
    2: (0, 255, 0),
    3: (255, 0, 0),
    4: (0, 128, 0),
}

def decode_segmentation(segmentation, class_colors):
    h, w = segmentation.shape
    color_image = np.zeros((h, w, 3), dtype=np.uint8)
    for class_id, color in class_colors.items():
        color_image[segmentation == class_id] = color
    return color_image[:, :, ::-1]  # BGR -> RGB

def preprocess_image(image_path, input_size=(512, 1024)):
    img = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    img_tensor = transform(img).unsqueeze(0).numpy()
    return img_tensor, np.array(img)

# 模型加载
onnx_path = "bisenetv2_5_cls_v2.onnx"
session = ort.InferenceSession(onnx_path, providers=['CPUExecutionProvider'])

# ===== 合并图像预测 =====
merged_image_path = "merged_images.jpg"
merged_mask_path = "merged_masks.png"

input_tensor, merged_img = preprocess_image(merged_image_path)
merged_out = session.run(None, {session.get_inputs()[0].name: input_tensor})
merged_pred_mask = merged_out[0][0, 0].astype(np.uint8)
merged_pred_color = decode_segmentation(merged_pred_mask, class_colors)

merged_gt = cv2.imread(merged_mask_path, cv2.IMREAD_GRAYSCALE)
merged_gt = cv2.resize(merged_gt, (merged_pred_mask.shape[1], merged_pred_mask.shape[0]), interpolation=cv2.INTER_NEAREST)
merged_gt_color = decode_segmentation(merged_gt, class_colors)

# ===== 单张图像路径 =====
image_paths = [
    'segdata/images/test/2024-10-02_data_cleansing_9.jpg',
    'segdata/images/test/2024-10-02_data_cleansing_24.jpg',
    'segdata/images/test/2024-10-02_data_cleansing_37.jpg',
    'segdata/images/test/2024-10-02_data_cleansing_38.jpg',
]

mask_paths = [
    'segdata/labels/test/2024-10-02_data_cleansing_9.png',
    'segdata/labels/test/2024-10-02_data_cleansing_24.png',
    'segdata/labels/test/2024-10-02_data_cleansing_37.png',
    'segdata/labels/test/2024-10-02_data_cleansing_38.png',
]

# ===== 单张图像、标签、预测结果列表 =====
single_imgs = []
single_gt_colors = []
single_pred_colors = []

for img_path, mask_path in zip(image_paths, mask_paths):
    input_tensor, rgb_img = preprocess_image(img_path)
    single_imgs.append(rgb_img)

    # 读取并 decode mask
    gt_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    gt_mask = cv2.resize(gt_mask, (rgb_img.shape[1], rgb_img.shape[0]), interpolation=cv2.INTER_NEAREST)
    gt_color = decode_segmentation(gt_mask, class_colors)
    single_gt_colors.append(gt_color)

    # 推理并 decode 预测
    output = session.run(None, {session.get_inputs()[0].name: input_tensor})
    pred_mask = output[0][0, 0].astype(np.uint8)
    pred_color = decode_segmentation(pred_mask, class_colors)
    single_pred_colors.append(pred_color)

plt.figure(figsize=(18, 14))

# 第一行: merged image, gt, pred + 一个空格
plt.subplot(4, 4, 1)
plt.imshow(merged_img)
plt.title("Merged Image")
plt.axis('off')

plt.subplot(4, 4, 2)
plt.imshow(merged_gt_color)
plt.title("Merged GT")
plt.axis('off')

plt.subplot(4, 4, 3)
plt.imshow(merged_pred_color)
plt.title("Merged Prediction")
plt.axis('off')

plt.subplot(4, 4, 4)
plt.axis('off')  # 填充空位

# 第二行: 4张单图原图
for i in range(4):
    plt.subplot(4, 4, 5 + i)
    plt.imshow(single_imgs[i])
    plt.title(f"Image {i+1}")
    plt.axis('off')

# 第三行: 4张标签
for i in range(4):
    plt.subplot(4, 4, 9 + i)
    plt.imshow(single_gt_colors[i])
    plt.title(f"GT {i+1}")
    plt.axis('off')

# 第四行: 4张预测
for i in range(4):
    plt.subplot(4, 4, 13 + i)
    plt.imshow(single_pred_colors[i])
    plt.title(f"Pred {i+1}")
    plt.axis('off')

plt.subplots_adjust(left=0.02, right=0.98, top=0.95, bottom=0.02, wspace=0.02, hspace=0.03)
plt.show()


In [ ]:
import cv2
import numpy as np

# 输入图像路径
image_paths = [
    'segdata/images/test/2024-10-02_data_cleansing_9.jpg',
    'segdata/images/test/2024-10-02_data_cleansing_24.jpg',
    'segdata/images/test/2024-10-02_data_cleansing_37.jpg',
    'segdata/images/test/2024-10-02_data_cleansing_38.jpg',
]

mask_paths = [
    'segdata/labels/test/2024-10-02_data_cleansing_9.png',
    'segdata/labels/test/2024-10-02_data_cleansing_24.png',
    'segdata/labels/test/2024-10-02_data_cleansing_37.png',
    'segdata/labels/test/2024-10-02_data_cleansing_38.png',
]

# 配置参数
target_width = 1920
target_height = 1080
gap = 0

# 计算缩放后的图像大小
img_w = (target_width - gap) // 2
img_h = (target_height - gap) // 2

# 读取并缩放图像
resized_images = []
resized_masks = []

for img_path, mask_path in zip(image_paths, mask_paths):
    img = cv2.imread(img_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # 假设mask是单通道图像

    if img is None:
        raise ValueError(f"Failed to load image: {img_path}")
    if mask is None:
        raise ValueError(f"Failed to load mask: {mask_path}")

    img_resized = cv2.resize(img, (img_w, img_h))
    mask_resized = cv2.resize(mask, (img_w, img_h), interpolation=cv2.INTER_NEAREST)

    resized_images.append(img_resized)
    resized_masks.append(mask_resized)

# 创建目标图像（黑色背景）
canvas_img = np.zeros((target_height, target_width, 3), dtype=np.uint8)
canvas_mask = np.zeros((target_height, target_width), dtype=np.uint8)

# 放置4张图像和mask
canvas_img[0:img_h, 0:img_w] = resized_images[0]
canvas_img[0:img_h, img_w + gap:img_w*2 + gap] = resized_images[1]
canvas_img[img_h + gap:img_h*2 + gap, 0:img_w] = resized_images[2]
canvas_img[img_h + gap:img_h*2 + gap, img_w + gap:img_w*2 + gap] = resized_images[3]

canvas_mask[0:img_h, 0:img_w] = resized_masks[0]
canvas_mask[0:img_h, img_w + gap:img_w*2 + gap] = resized_masks[1]
canvas_mask[img_h + gap:img_h*2 + gap, 0:img_w] = resized_masks[2]
canvas_mask[img_h + gap:img_h*2 + gap, img_w + gap:img_w*2 + gap] = resized_masks[3]

# 保存结果
cv2.imwrite("merged_images.jpg", canvas_img)
cv2.imwrite("merged_masks.png", canvas_mask)


In [ ]:
import onnxruntime as ort
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import torch
from torchvision import transforms

# 定义类别颜色（BGR）
class_colors = {
    0: (0, 0, 0),
    1: (0, 255, 255),
    2: (0, 255, 0),
    3: (255, 0, 0),
    4: (0, 128, 0),
}

def preprocess_image(image_path, input_size=(512, 1024)):
    img = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    img_tensor = transform(img).unsqueeze(0).numpy()  # 转为 numpy
    return img_tensor, np.array(img)

def decode_segmentation(segmentation, class_colors):
    h, w = segmentation.shape
    color_image = np.zeros((h, w, 3), dtype=np.uint8)
    for class_id, color in class_colors.items():
        color_image[segmentation == class_id] = color
    return color_image[:, :, ::-1]  # BGR -> RGB

# 加载 ONNX 模型
onnx_path = "bisenetv2_5_cls_v2.onnx"
session = ort.InferenceSession(onnx_path, providers=['CPUExecutionProvider'])

# 输入图片路径
# image_path = "segdata/images/test/2024-10-02_data_cleansing_9.jpg"
image_path = "merged_images.jpg"
gt_mask_path = "merged_masks.png"

input_tensor, original_img = preprocess_image(image_path)

# 推理
ort_inputs = {session.get_inputs()[0].name: input_tensor}
ort_outs = session.run(None, ort_inputs)

# 输出为 (1, 1, 512, 1024)，取出预测 mask
pred_mask = ort_outs[0][0, 0, :, :].astype(np.uint8)

# 读取 GT mask 并缩放到相同大小
gt_mask = cv2.imread(gt_mask_path, cv2.IMREAD_GRAYSCALE)
gt_mask = cv2.resize(gt_mask, (pred_mask.shape[1], pred_mask.shape[0]), interpolation=cv2.INTER_NEAREST)
gt_color = decode_segmentation(gt_mask, class_colors)


# 可视化
pred_color = decode_segmentation(pred_mask, class_colors)
# 显示结果
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(original_img)
plt.title("Original Image")
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(pred_color)
plt.title("ONNX Prediction")
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(gt_color)
plt.title("Ground Truth Mask")
plt.axis('off')

plt.tight_layout()
plt.show()